In [48]:
from input.data_read_write import DataReaderAndWriter

In [49]:
data_holder = DataReaderAndWriter()
data_holder.parse_from_pickle_file("with_problems.pkl")
data = data_holder.data

# Create vocab of tokenizer values

In [50]:
from io import BytesIO
from tokenize import tokenize
from collections import Counter
from tqdm.autonotebook import tqdm

In [51]:
c = Counter()

for d in tqdm(data):
    src_code = d['src']
    tokens= tokenize(BytesIO(src_code.encode('utf-8')).readline)
    for _, tokval, _, _, _ in tokens:
        c.update([tokval])
c.most_common(20)


100%|██████████| 6623/6623 [00:38<00:00, 172.05it/s]


[('\n', 1164200),
 (',', 642672),
 ('(', 626956),
 (')', 626956),
 ('.', 600472),
 ('=', 390064),
 (':', 232380),
 ('self', 197177),
 ('', 188956),
 ('[', 136408),
 (']', 136408),
 ('def', 69717),
 ('        ', 66507),
 ('1', 53460),
 ('            ', 51635),
 ('if', 48677),
 ('0', 43515),
 ('import', 38006),
 ('torch', 37223),
 ('return', 36465)]

In [52]:
vocab_dict= {}
max_vocab_size = 1000
for i, vocab in enumerate(c.most_common(max_vocab_size)):
    vocab_dict[i] = vocab

vocab_holder = DataReaderAndWriter()
vocab_holder.data = vocab_dict
vocab_holder.dump_to_file(f"vocab_{max_vocab_size}.pkl")

# Preprocess the data

Approach: use a tokenizer, use type and value embedded.

In [53]:
def token_type_to_number(token_type):
    return token_type    

In [54]:
vocab = vocab_holder = DataReaderAndWriter()
vocab_holder.parse_from_pickle_file("vocab_1000.pkl")
vocab = vocab_holder.data 

## Turn original data structure into 
\[token_type, token_value, ...\]\[0|1\]
use a sliding window to process all file from beginning to end

In [55]:
def code_to_windowed_data(src_code, problem_line_numbers):
    pass

In [68]:
from rules.rule_processor import run_rules_on_dataset

#load small dataset
test_data_holder = DataReaderAndWriter()
test_data_holder.load_source_file_from("test_file/")


processed_data = run_rules_on_dataset(test_data_holder.data)

data = run_rules_on_dataset(data)
data

100%|██████████| 1/1 [00:00<00:00, 477.82it/s]




[{'file_path': 'test_file/return_none_test.py',
  'src': 'import os\nimport sys\n\ndef hello0(bla, asd, ds):\n    return bla\n\n\n\ndef hello(bla, asd, ds):\n    return None\n\ndef hello2(bla, asd, ds):\n    return asd\n\ndef hello3(bla, asd, ds):\n    return ds\n\n',
  'problems': [{'type': 'RETURN_NULL', 'line_number': 10, 'col_offset': 4}]}]

In [63]:
for d in data:
    problem_line_numbers = [l['line_number'] for l in d['problems'] if l['type'] == "RETURN_NULL"]
    

KeyError: 'problems'